## Pipeline Example

1. Create input and processor chain

In [4]:
from logprep.factory import Factory
from logprep.ng.abc.processor import Processor
from logprep.event.log_event import LogEvent
from logprep.event.event_state import EventStateType
from logprep.ng.pipeline import Pipeline


input_connector = iter(
    [
        LogEvent({"message": "Log message 1"}, original=b"", state=EventStateType.RECEIVED),
        LogEvent({"message": "Log message 2"}, original=b"", state=EventStateType.RECEIVED),
        LogEvent({"user": {"name": "John Doe"}}, original=b"", state=EventStateType.RECEIVED),
    ]
)

def get_processors() -> list[Processor]:
    processors = [
        Factory.create(
            {
                "processor": {
                    "type": "ng_generic_adder",
                    "rules": [
                        {
                            "filter": "*",
                            "generic_adder": {"add": {"event.tags": "generic added tag"}},
                        }
                    ],
                }
            }
        ),
        Factory.create(
            {
                "pseudo_this": {
                    "type": "ng_pseudonymizer",
                    "pubkey_analyst": "../../../../../examples/exampledata/rules/pseudonymizer/example_analyst_pub.pem",
                    "pubkey_depseudo": "../../../../../examples/exampledata/rules/pseudonymizer/example_depseudo_pub.pem",
                    "regex_mapping": "../../../../../examples/exampledata/rules/pseudonymizer/regex_mapping.yml",
                    "hash_salt": "a_secret_tasty_ingredient",
                    "outputs": [{"opensearch": "pseudonyms"}],
                    "rules": [
                        {
                            "filter": "user.name",
                            "pseudonymizer": {
                                "id": "pseudonymizer-1a3c69b2-5d54-4b6b-ab07-c7ddbea7917c",
                                "mapping": {"user.name": "RE_WHOLE_FIELD"},
                            },
                        }
                    ],
                    "max_cached_pseudonyms": 1000000,
                }
            }
        ),
    ]
    for processor in processors:
        processor.setup()
    return processors

processors = get_processors()


2. create the pipeline

In [5]:

pipeline = Pipeline(input_connector, processors)

3. run the pipeline

In [6]:
for event in pipeline:
    print(f"Processed event: {event.data}")
    print(f"Event state: {event.state}")
    print(f"generated extra_data: {event.extra_data}")
    print("-" * 40)

Processed event: {'message': 'Log message 1', 'event': {'tags': 'generic added tag'}}
Event state: processed
generated extra_data: []
----------------------------------------
Processed event: {'message': 'Log message 2', 'event': {'tags': 'generic added tag'}}
Event state: processed
generated extra_data: []
----------------------------------------
Processed event: {'user': {'name': '<pseudonym:12f7a0505314df2259513546a7e3da518098b6999443ff681a56fb752afa998b>'}, 'event': {'tags': 'generic added tag'}}
Event state: processed
generated extra_data: [PseudonymEvent(data={'pseudonym': '12f7a0505314df2259513546a7e3da518098b6999443ff681a56fb752afa998b', 'origin': 'idhyPRJcrPrMsNZW70q6XQGkRK9/ue/G73g3b2A0kb30HsiBjt0z5/QeZPsUCMXwyzA5xOJjCt9J/505HVTvaezDLYGmmyQ9xc6SWwEXo9ZcUOLX2smCwGvzHfxtll5BuxaTnEQFdI28FRWLoJYNh8sj4L7TEhsAG6ALTtsMcbw=:opR2Cs952QlCWT8960QakQ==:IMtvLai6fI3BNwi5mByge4swaQMAg90wtumuuBeGGqYC4vRUHtE0kZ3igCVgYpNjEFYqX0JAtq4odMFfEv9ePGhCi0D1gUfNYQyyErinvpARRIGQyxPPunDXVtGiwwGvyXyMEohcd